# Dataframes con PySpark

En este notebook aprenderemos cómo crear DataFrames a partir de colecciones de Python y de ficheros externos.

**Instalaciones previas, si fuesen necesarias.**

In [ ]:
# Instalar Java si no está instalado
!apt install -y openjdk-21-jdk

In [ ]:
# Definir la variable de entorno JAVA_HOME si fuese necesario

# En nuestro ordenador personal, si no esta definida la variable JAVA_HOME, deberemos indicarla
# Para sistemas basados en Debian/Ubuntu, si tenemos instalada la version 21 de Java, seria:
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
# En Windows, la ruta puede ser algo como: "C:\\Program Files\\Java\\jdk-21"
# import os
# os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-21"
# Si ya esta definida, no es necesario hacer nada
# import os
# os.environ["JAVA_HOME"]

# En Google Colab no es necesario hacer nada

In [ ]:
# Instalar PySpark si no está instalado
!pip install pyspark

Comenzamos importando las librerías necesarias y creando una sesión de Spark.

In [ ]:
from pyspark.sql import SparkSession

# Crear sesión de Spark
spark = SparkSession.builder.appName("Analisis Deportivo con Spark").getOrCreate()

### Ejemplo 3

Vamos a crear un Dataframe a partir de una colección de Python. 

In [4]:
# Datos simulados: equipo, goles, posesión, pases
data = [("EquipoA", 2, 55.0, 480), ("EquipoB", 1, 45.0, 390)]

# Crear DataFrame a partir de la lista indicando las columnas
df = spark.createDataFrame(data, ["equipo", "goles", "posesion", "pases"])

# Mostrar el DataFrame
df.show()

+-------+-----+--------+-----+
| equipo|goles|posesion|pases|
+-------+-----+--------+-----+
|EquipoA|    2|    55.0|  480|
|EquipoB|    1|    45.0|  390|
+-------+-----+--------+-----+



### Ejemplo 4

En este ejemplo vamos a crear un Dataframe a partir de un fichero CSV con datos de jugadores de fútbol. El fichero no tiene cabecera, por lo que debemos definir el esquema de los datos.

**Si estamos trabajando en local** con Jupyter Notebook/Lab, accedemos directamente utilizando la carpeta `data/`.

In [5]:
file_path = "data/wordcount_data.txt"

**Si estamos trabajando en Google Colab**, montamos una carpeta de Google Drive que contenga nuestros datos.

In [ ]:
# Montamos la carpeta (nos pedirá permisos)
from google.colab import drive
drive.mount('/content/drive')
# Crea un atajo llamado 'workspace' en la carpeta /content (dará un pequeño error si ya existe)
!ln -s "/content/drive/MyDrive/Colab Notebooks" "/content/workspace" >/dev/null 2>&1
# Ya podemos acceder a los ficheros, por ejemplo:
file_path = "/content/workspace/data/wordcount_data.txt"

Leemos el fichero de texto y creamos un DataFrame.

In [6]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

# Definir el esquema manualmente
schema = StructType([
    StructField("Jugador", StringType(), True),
    StructField("Equipo", StringType(), True),
    StructField("Velocidad", DoubleType(), True),
    StructField("Goles", IntegerType(), True),
    StructField("Pases", IntegerType(), True)
])

# Leer el archivo CSV con el esquema
df = spark.read.csv("data/jugadores.csv", header=False, schema=schema)

# Mostrar el contenido
df.show()

# Mostrar el esquema inferido
df.printSchema()

+--------+-------+---------+-----+-----+
| Jugador| Equipo|Velocidad|Goles|Pases|
+--------+-------+---------+-----+-----+
|Jugador1|EquipoA|     28.5|    2|  480|
|Jugador2|EquipoA|     30.2|    1|  510|
|Jugador3|EquipoB|     26.7|    0|  390|
+--------+-------+---------+-----+-----+

root
 |-- Jugador: string (nullable = true)
 |-- Equipo: string (nullable = true)
 |-- Velocidad: double (nullable = true)
 |-- Goles: integer (nullable = true)
 |-- Pases: integer (nullable = true)



Finalmente, podemos cerrar la sesión de Spark cuando ya no la necesitemos. Si lo hacemos, habrá que ejecutar todo el notebook de nuevo para volver a crearla.

In [7]:
spark.stop()